In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [0]:
path = "drive/My Drive/MSc BD & BA/Period 3/NLPy/data.json"

In [0]:
import json
import time

import pandas as pd
import numpy as np
from tqdm import tqdm

from pprint import pprint
from flair.data import Sentence
from flair.embeddings import BertEmbeddings, FlairEmbeddings, StackedEmbeddings


import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
with open(path, 'r') as f:
    data = json.load(f)

In [0]:
# Utils

def batch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def make_input(dictionnary, key):
    line = dictionnary[key]
    
    message =  '[SOS] {} [EOS]'.format(line['message'])
    
    ans = '[SANS] {} [EANS]'.format(line['num_answers'])
    cat = '[SCAT] {} [ECAT]'.format(line['category'])
    country = '[SCOUNT] {} [ECOUNT]'.format(line['country'])
    member = '[SMEM] {} [EMEM]'.format(line['member'])
    solved = '[SSOL] {} [ESOL]'.format(line['is_solved'])
    state = '[SSTAT] {} [ESTAT]'.format(line['visits'])
    title = '[SOT] {} [EOT]'.format(line['titre'])
    topic = '[STOP] {} [ETOP]'.format(line['topic_id'])
    visits = '[SVIS] {} [EVIS]'.format(line['visits'])
    votes = '[SVOT] {} [EVOT]'.format(line['votes'])
    
    inp = message + cat + country + member + solved + state
    inp = inp + title + topic+ visits + votes
    
    return inp

In [0]:
# init Flair embeddings
flair_forward_embedding = FlairEmbeddings('multi-forward')
flair_backward_embedding = FlairEmbeddings('multi-backward')

# init multilingual BERT
bert_embedding = BertEmbeddings('bert-base-multilingual-cased')

stacked_embeddings = StackedEmbeddings(
    embeddings=[flair_forward_embedding, flair_backward_embedding, 
                bert_embedding
               ]
    )

def embedding(sentence):
    stacked_embeddings.embed(sentence)
    return torch.Tensor([list(word.embedding) for word in sentence])

In [0]:
class LSTMClassifier(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size, use_gpu):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.use_gpu = use_gpu
        
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        if self.use_gpu:
            h0 = Variable(torch.zeros(1, self.batch_size, self.hidden_dim).cuda())
            c0 = Variable(torch.zeros(1, self.batch_size, self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(1, self.batch_size, self.hidden_dim))
            c0 = Variable(torch.zeros(1, self.batch_size, self.hidden_dim))
        return (h0, c0)

    def forward(self, sentence):
        embeds = sentence
        x = embeds.view(len(sentence), self.batch_size, -1)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y  = self.hidden2label(lstm_out[-1])
        return y

      
model = LSTMClassifier(embedding_dim = 7168, 
                       hidden_dim = 128, 
                       label_size = 1, 
                       batch_size = 1, 
                       use_gpu = True).to(device)

In [20]:
model.apply(init_weights)

optimizer = optim.Adam(model.parameters())

criterion = nn.BCELoss()

print("The model has {} trainable parameters".format(count_parameters(model)))

The model has 3736705 trainable parameters


In [0]:
def stochastic_train(model, optimizer, criterion, clip, data_source):
    
    losses = {}
    mod_count = 0
    
    model.train()
    
    for key in data_source.keys():
        
        start_time = time.time()
        
        src = make_input(data_source, key)
        trg = int(data_source[key]['is_best_answer'])
        
        # Got the minibatch
        optimizer.zero_grad()
        
        # Launching BERT embedding
        src = embedding(Sentence(src)).to(device)
        trg = torch.Tensor([trg]).to(device)
        
        # Launching LSTM forward
        output = model(src).squeeze_(0)
        
        model.zero_grad()
        
        
        loss = criterion(torch.sigmoid(output), trg)
        print('Loss is: ',loss.item())
        print('Launching Backprop')
        

        loss.backward(retain_graph=True)
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        end_time = time.time()
        batch_mins, batch_secs = batch_time(start_time, end_time)
        
        print('Time: {}:{}\nLoss: {}'.format(batch_mins,batch_secs,loss.item()))
        losses[key] = loss.item()
        
        
        if int(key)+1% 100 == 0:
            with ("drive/My Drive/MSc BD & BA/Period 3/NLPy/results.json", 'w') as f:
                json.dump(losses, f)
            torch.save(model.state_dict(), 
                       'drive/My Drive/MSc BD & BA/Period 3/NLPy/model{}.pt'.format(
                       mod_count))
            mod_count +=1
        
    return losses, mod_count

In [26]:
N_EPOCHS = 1
CLIP = 1
LOAD = False

best_valid_loss = float('inf')
train_loss, mod_count = stochastic_train(model, optimizer, criterion, CLIP, data)


if LOAD:
    model.load_state_dict(torch.load('drive/My Drive/MSc BD & BA/Period 3/NLPy/model{}.pt'.format(
        mod_count)))

Got the minibatch
Launching BERT embedding
Launching LSTM forward
Loss is:  0.20415067672729492
Launching Backprop
Time: 0:1
Loss: 0.20415067672729492
Got the minibatch
Launching BERT embedding
Launching LSTM forward
Loss is:  0.12183970212936401
Launching Backprop
Time: 0:1
Loss: 0.12183970212936401
Got the minibatch
Launching BERT embedding
Launching LSTM forward
Loss is:  0.07801752537488937
Launching Backprop
Time: 0:2
Loss: 0.07801752537488937
Got the minibatch
Launching BERT embedding
Launching LSTM forward
Loss is:  0.06253203004598618
Launching Backprop
Time: 0:2
Loss: 0.06253203004598618
Got the minibatch
Launching BERT embedding


KeyboardInterrupt: ignored